In [14]:
using LinearAlgebra, DataFrames

# Problem 1
# M = [2 3+im 1 -3;
#      3-im 1 -5-(2*im) 2*im;
#      1 -5-(2*im) 2 -2;
#      -3 -(2*im) -2 -3]

# Problem 2
# M = [2*im 2 1-2*im -5*im;
#      -2 -3*im 2+im 0;
#      -1-2*im -2+im 0 -5;
#      -5*im 0 5 -4*im]

# Problem 3
# M = [2 2 5;
#      -1 1 3;
#      5 0 -2]

# Problem 4
M = [2 5 1;
     -1 1 3;
     3 0 4]

3×3 Matrix{Int64}:
  2  5  1
 -1  1  3
  3  0  4

In [15]:
# Complex conjugate transpose of M
M_H = copy(M)

function complex_conjugate_transpose(M)
    M_H = copy(M)
    for i in 1:size(M, 1)
        for j in 1:size(M, 2)
            M_H[i, j] = conj(M[j, i])
        end
    end
    return M_H
end

M_H = complex_conjugate_transpose(M)

# Show M_H as a dataframe
M_H_df = DataFrame(M_H, :auto)
show(M_H_df, allrows=true, allcols=true)

3×3 DataFrame
 Row │ x1     x2     x3    
     │ Int64  Int64  Int64 
─────┼─────────────────────
   1 │     2     -1      3
   2 │     5      1      0
   3 │     1      3      4

In [16]:
# Check if M is Hermitian from its eigenvalues
eigvals(M)

3-element Vector{ComplexF64}:
 0.38272559780771487 - 3.3288554009364555im
 0.38272559780771487 + 3.3288554009364555im
    6.23454880438457 + 0.0im

In [17]:
# Check if M is Hermitian
herm = ishermitian(M)

false

In [18]:
# Check if M is skew Hermitian
skew = ishermitian(im * M)

false

In [19]:
uni_df = DataFrame(M_H * M, :auto)
show(uni_df, allrows=true, allcols=true)

3×3 DataFrame
 Row │ x1     x2     x3    
     │ Int64  Int64  Int64 
─────┼─────────────────────
   1 │    14      9     11
   2 │     9     26      8
   3 │    11      8     26

In [20]:
# Check if M is unitary
unitary = (M_H * M == I(4))

false

In [21]:
# Diagonalize M, round eigenvalues less than 1e-10 to 0
D, P = eigen(M)
P_inv = inv(P)
D_matrix = Diagonal(D)

# Display D as a dataframe
D_df = DataFrame(D_matrix, :auto)
show(D_df, allrows=true, allcols=true)

3×3 DataFrame
 Row │ x1                  x2                  x3            
     │ Complex…            Complex…            Complex…      
─────┼───────────────────────────────────────────────────────
   1 │ 0.382726-3.32886im       0.0+0.0im          0.0+0.0im
   2 │      0.0+0.0im      0.382726+3.32886im      0.0+0.0im
   3 │      0.0+0.0im           0.0+0.0im      6.23455+0.0im

In [22]:
P_df = DataFrame(P, :auto)
show(P_df, allrows=true, allcols=true)

3×3 DataFrame
 Row │ x1                   x2                   x3              
     │ Complex…             Complex…             Complex…        
─────┼───────────────────────────────────────────────────────────
   1 │ -0.70939-0.0im       -0.70939+0.0im       -0.564603+0.0im
   2 │ 0.165745+0.530922im  0.165745-0.530922im  -0.326566+0.0im
   3 │ 0.318554-0.293155im  0.318554+0.293155im  -0.758009+0.0im

In [23]:
P_inv_df = DataFrame(P_inv, :auto)
show(P_inv_df, allrows=true, allcols=true)

3×3 DataFrame
 Row │ x1                       x2                      x3                     
     │ Complex…                 Complex…                Complex…               
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ -0.522939-0.0226809im     0.173743-0.753246im    0.314659+0.341407im
   2 │ -0.522939+0.0226809im     0.173743+0.753246im    0.314659-0.341407im
   3 │ -0.457075-6.76159e-18im  -0.436594-7.3677e-18im   -0.7907+1.47603e-17im

In [24]:
# Calculate M^n
n = 100
M_n = P * (D_matrix^n) * P_inv

# Show M_n as a dataframe
M_n_df = DataFrame(M_n, :auto)
show(M_n_df, allrows=true, allcols=true)

3×3 DataFrame
 Row │ x1                       x2                       x3                      
     │ Complex…                 Complex…                 Complex…                
─────┼───────────────────────────────────────────────────────────────────────────
   1 │ 7.80249e78+1.15424e62im  7.45288e78+1.2577e62im   1.34976e79-2.51966e62im
   2 │ 4.51295e78+6.67609e61im  4.31074e78+7.27455e61im  7.80702e78-1.45737e62im
   3 │ 1.04753e79+1.54962e62im  1.00059e79+1.68853e62im  1.81213e79-3.38277e62im

In [25]:
"""
    exp_spectral(M; atol=1e-10, rtol=1e-10)

Compute exp(M) using the spectral route chosen by classification:
- Hermitian:  M = U*Λ*U', exp(M) = U*exp.(Λ)*U'
- Skew-Hermitian: M = iH with H Hermitian, so exp(M) = U*exp.(i*Λ)*U'
- Normal (else): diagonalizable by eigenvectors V, exp(M) = V*exp.(Λ)*inv(V)
Falls back to LinearAlgebra.exp(M) if something goes sideways.
Returns (expM, method::String).
"""
function exp_spectral(M; atol=1e-10, rtol=1e-10)
    try
        if ishermitian(M)
            F = eigen(Hermitian(M))
            return (F.vectors * Diagonal(exp.(F.values)) * F.vectors', "Hermitian spectral")
        elseif skew
            H = (-1im) * M                         # H Hermitian
            F = eigen(Hermitian(H))
            return (F.vectors * Diagonal(exp.(1im .* F.values)) * F.vectors', "Skew-Hermitian via H")
        elseif is_normal(M; atol=atol, rtol=rtol)
            # Normal ⇒ diagonalizable. eigen(M) is fine; V needn't be unitary.
            F = eigen(M)
            return (F.vectors * Diagonal(exp.(F.values)) * inv(F.vectors), "Normal spectral")
        else
            # Not safely spectral; fall back
            return (exp(M), "Fallback: exp(M)")
        end
    catch
        # Any numerical hiccup → robust fallback
        return (exp(M), "Fallback (caught error): exp(M)")
    end
end

# --------------------------
# Drive it
# --------------------------
expM, how = exp_spectral(M)
println("Method used: ", how)

# sanity check against built-in exp
println("‖exp_spectral - exp(M)‖₂ = ", norm(expM - exp(M)))

# Show exp(M) as a dataframe
expM_df = DataFrame(expM, :auto)
show(expM_df, allrows=true, allcols=true)

Method used: Fallback (caught error): exp(M)
‖exp_spectral - exp(M)‖₂ = 0.0
3×3 DataFrame
 Row │ x1        x2        x3      
     │ Float64   Float64   Float64 
─────┼─────────────────────────────
   1 │ 130.554   125.797   228.487
   2 │  76.5043   71.5067  131.958
   3 │ 177.142   169.439   305.128

In [26]:
# u = ComplexF64[1, 0, -3, 2]
# println("‖u‖₂ = ", norm(u), "  ‖e^M u‖₂ = ", norm(expM * u))